In [17]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

In [26]:
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import VGG16

In [5]:
# create the base pre-trained model
#include_top = False since we will adapt a dense layer for CIFAR-10 Classification
# base_model = InceptionV3(weights='imagenet', include_top=False)

In [40]:
from keras.layers import Input
# this could also be the output a different Keras model or layer
input_tensor = Input(shape=(32, 32, 3))  # this assumes K.image_data_format() == 'channels_last'

base_model = VGG19(input_tensor=input_tensor, weights='imagenet', include_top=False)


In [41]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:22]:
   layer.trainable = False

In [42]:
# add a global spatial average pooling layer
x = base_model.output

x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# and a logistic layer -- let's say we have 10 classes
predictions = Dense(10, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [46]:
import os
import time
os.chdir('/notebooks/src/CIFAR-10/')

In [59]:
from lib.load_images import load_data
(X_train, y_train), (X_test, y_test) = load_data()
epochs = 1
batch_size=32
nb_train_samples = X_train.shape[0]
nb_validation_samples = X_test.shape[0]

In [60]:
nb_train_samples

50000

In [61]:
nb_validation_samples

10000

In [62]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping 
# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

test_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

train_generator = train_datagen.flow(
X_train,
batch_size = batch_size)

validation_generator = test_datagen.flow(
X_test)

# Save the model according to the conditions  
# checkpoint = ModelCheckpoint("vgg19_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
# early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

In [66]:
# Train the model 
model.fit_generator(
train_generator,
samples_per_epoch = nb_train_samples,
epochs = epochs,
validation_data = validation_generator,
nb_val_samples = nb_validation_samples,
callbacks = [checkpoint, early]
)

Epoch 1/1


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=1562, epochs=1, callbacks=[<keras.ca..., validation_steps=10000)`
  


ValueError: Output of generator should be a tuple `(x, y, sample_weight)` or `(x, y)`. Found: [[[[ 0.00329104  0.00335256  0.0023837 ]
   [ 0.00341407  0.00319877  0.0018762 ]
   [ 0.00333718  0.00306036  0.00230681]
   ..., 
   [ 0.00190696  0.00219915  0.00109189]
   [ 0.00190696  0.00219915  0.00109189]
   [ 0.00190696  0.00219915  0.00109189]]

  [[ 0.00329104  0.00335256  0.0023837 ]
   [ 0.00329104  0.00335256  0.0023837 ]
   [ 0.00341407  0.00319877  0.0018762 ]
   ..., 
   [ 0.00190696  0.00219915  0.00109189]
   [ 0.00190696  0.00219915  0.00109189]
   [ 0.00190696  0.00219915  0.00109189]]

  [[ 0.00253749  0.00241446  0.00172241]
   [ 0.00329104  0.00335256  0.0023837 ]
   [ 0.00341407  0.00319877  0.0018762 ]
   ..., 
   [ 0.00190696  0.00219915  0.00109189]
   [ 0.00190696  0.00219915  0.00109189]
   [ 0.00190696  0.00219915  0.00109189]]

  ..., 
  [[ 0.0023837   0.00186082  0.00159938]
   [ 0.00267589  0.00219915  0.00178393]
   [ 0.00249135  0.00204537  0.00167628]
   .

In [ ]:
# start = time.time()
# # Fit the model on the batches generated by datagen.flow().
# model_info = model.fit(X_train, y_train,
#                     epochs=epoch,
#                     validation_data=(X_test, y_test))

# end = time.time()


In [37]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

In [39]:
# # at this point, the top layers are well trained and we can start fine-tuning
# # convolutional layers from inception V3. We will freeze the bottom N layers
# # and train the remaining top layers.

# # let's visualize layer names and layer indices to see how many layers
# # we should freeze:
# for i, layer in enumerate(base_model.layers):
#    print(i, layer.name)

In [11]:
# # we chose to train the top 2 inception blocks, i.e. we will freeze
# # the first 249 layers and unfreeze the rest:
# for layer in model.layers[:22]:
#    layer.trainable = False
# for layer in model.layers[22:]:
#    layer.trainable = True

In [13]:
# from keras.preprocessing.image import ImageDataGenerator
# datagen = ImageDataGenerator(
#     featurewise_center=False,  # set input mean to 0 over the dataset
#     samplewise_center=False,  # set each sample mean to 0
#     featurewise_std_normalization=False,  # divide inputs by std of the dataset
#     samplewise_std_normalization=False,  # divide each input by its std
#     zca_whitening=False,  # apply ZCA whitening
#     rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
#     width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
#     height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
#     horizontal_flip=True,  # randomly flip images
#     vertical_flip=False)  # randomly flip images

# # Compute quantities required for feature-wise normalization
# # (std, mean, and principal components if ZCA whitening is applied).
# datagen.fit(X_train)


In [38]:
# # we need to recompile the model for these modifications to take effect
# # we use SGD with a low learning rate
# from keras.optimizers import SGD, rmsprop
# # RMSprop optimizer
# opt = rmsprop(lr=0.0001, decay=1e-6)
# # SGD optimizer
# # opt = SGD(lr=0.0001, momentum=0.9)

# model.compile(optimizer=opt, loss='categorical_crossentropy')

# # we train our model again (this time fine-tuning the top 2 inception blocks
# # alongside the top Dense layers

# start = time.time()

# # model_info = model.fit_generator(generator(X_train, y_train, batch_size=32),
# #                     epochs=epoch,
# #                     validation_data=(X_test, y_test))
# # Fit the model on the batches generated by datagen.flow().
# model_info = model.fit_generator(datagen.flow(X_train, y_train,
#                                  batch_size=32),
#                     steps_per_epoch=X_train.shape[0] // 32,
#                     epochs=1,
#                     validation_data=(X_test, y_test))
# end = time.time()


In [ ]:
import matplotlib.pyplot as plt
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
    axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

In [ ]:
import numpy as np
def accuracy(test_x, test_y, model):
    result = model.predict(test_x)
    predicted_class = np.argmax(result, axis=1)
    true_class = np.argmax(test_y, axis=1)
    num_correct = np.sum(predicted_class == true_class) 
    accuracy = float(num_correct)/result.shape[0]
    return (accuracy * 100)
